In [ ]:
#import the necessary libraries
import numpy as np
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
import os
from keras import models 
from keras import layers

In [ ]:
#Mount Google drive
#Run this cell only if your data (npy files of LGG and HGG reside on Google 
#drive)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Set base_path to the location where the data and results of your project
#reside
base_path = '/content/gdrive/MyDrive/HPT/'

In [ ]:
# Image shape set to constant for further use
# 240, 240 is the size of a slice in BraTS dataset. Same image/slice
# is copied to the 3 channels. We need to have 3 channels because we
# are using pre-trained ResNet50 (or its variant)
IMG_SHAPE = (240, 240, 3)

#Preparing TrainX and TrainY
Loading HGG and LGG data stored in .npy files. Creating their labels: 0 for LGG and 1 for HGG. Finally, the data and the corresponding labels will be shuffled randomly.

In [ ]:
HGG_cases = 19496
LGG_cases = 4926
Total_cases = HGG_cases + LGG_cases

In [ ]:
#creating a NumPy array for holding all the training data (TrainX)
TrainX = np.zeros((Total_cases, 240 , 240, 3), dtype=np.float16)

In [ ]:
print (TrainX.dtype)

float16


In [ ]:
#function to load HGG cases stored in BraTS2020_Tumorous_HGG_T1_f16.npy
def read_HGG():
  HGG_data_one_channel = np.load(base_path + 'Datasets/BraTS2020/BraTS2020_Tumorous_HGG_T1_f16.npy')
  print (HGG_data_one_channel.shape)
  print (HGG_data_one_channel.dtype)

  for i in range (HGG_data_one_channel.shape[0]):
    TrainX[i, :, :, 0] = HGG_data_one_channel[i, :, :]
    TrainX[i, :, :, 1] = HGG_data_one_channel[i, :, :]
    TrainX[i, :, :, 2] = HGG_data_one_channel[i, :, :]

In [ ]:
#function to load LGG cases stored in BraTS2020_Tumorous_LGG_T1_f16.npy
def read_LGG():
  LGG_data_one_channel = np.load(base_path + 'Datasets/BraTS2020/BraTS2020_Tumorous_LGG_T1_f16.npy')
  print (LGG_data_one_channel.shape)
  print (LGG_data_one_channel.dtype)

  for i in range (LGG_data_one_channel.shape[0]):
    TrainX[i + HGG_cases, :, :, 0] = LGG_data_one_channel[i, :, :]
    TrainX[i + HGG_cases, :, :, 1] = LGG_data_one_channel[i, :, :]
    TrainX[i + HGG_cases, :, :, 2] = LGG_data_one_channel[i, :, :]

In [ ]:
#call the function read_HGG() to load HGG data to TrainX
read_HGG()

(19496, 240, 240)
float16


In [ ]:
#call the function read_LGG() to load LGG data to TrainX
read_LGG()

(4926, 240, 240)
float16


In [ ]:
#function to define labels. i.e. 1 for HGG and 0 LGG cases
def define_labels():
  HGG_labels = np.ones(shape=(HGG_cases,1), dtype='uint8')
  LGG_labels = np.zeros(shape=(LGG_cases,1), dtype='uint8')

  return (np.concatenate((HGG_labels, LGG_labels), axis=0))



In [ ]:
#Call the function to create labels and store in TrainY 
TrainY = define_labels()

In [ ]:
#Printing the shape of TrainX and TrainY
print (TrainX.shape)
print (TrainY.shape)

(24422, 240, 240, 3)
(24422, 1)


In [ ]:
#Shuffle the data in TrainX and TrainY
p = np.random.permutation(TrainX.shape[0])
TrainX = TrainX[p]
TrainY = TrainY[p]



# Phase 1 of DAPT (Start for the First Time)
Run the following cells only when starting DAPT (phase 1) for a particular 
strategy for the first time. **DO NOT** run the following cells if you are
resuming phase 1 of DAPT after some epochs.

#All the layers of the architecture will be frozen except the last layer (sigmoid) for 1000 epochs

In [ ]:
# Loading the convolution base of ResNet50 with ImageNet pre-trained 
# weights
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, 
                                               include_top=False, 
                                               weights="imagenet")

94781440/94765736 [==============================] - 0s 0us/step


In [ ]:
# Printing the summary of the base_model and its layers
base_model.summary()
print (len(base_model.layers))

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 240, 240, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 246, 246, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 120, 120, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 120, 120, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
def transfuse_weights(simple_architecture):
  for i, layer in enumerate(base_model.layers):
  Flag = 0
  for j, trans_layer in enumerate(simple_architecture.layers): 
    if layer.name==trans_layer.name:
      #counter = counter + 1
      #Flag = 1
      simple_architecture.layers[j].set_weights(base_model.layers[i].get_weights())
    if Flag == 0:
      print (i, layer.name)

  return simple_architecture

In [ ]:
#This function creates an architecture based on the first one block
#of the ResNet50 architecture and trans
def create_first_one_block_architecture():

  img_input = layers.Input(shape=IMG_SHAPE) #input layer
  x = img_input
  for i, layer in enumerate(base_model.layers):
    if i >= 1 and i<= 12:
      x = layer(x)
      if i == 6:
        temp1 = x
    elif i == 13:
      shortcut1 = layer(temp1)
    elif i == 14:
      x = layer(x)
    elif i == 15:
      shortcut1 = layer(shortcut1)
    elif i == 16:
      x = layer(x)
    elif i == 17:
      x = layers.add([x, shortcut1])
    elif i >= 18 and i <= 26:
      x = layer(x)
      if i == 18:
        temp2 = x
    elif i == 27:
      x = layers.add([x, temp2])
    elif i >= 28 and i <= 36:
      x = layer(x)
      if i == 28:
        temp3 = x
    elif i == 37:
      x = layers.add([x, temp3]) 
    elif i==38:
      x = layer(x)

  x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
  x = layers.Dense(1, activation='sigmoid', name='fc')(x)
  DA_TF_F1B_model = models.Model(img_input, x, name='DA_TF_F1B')

  DA_TF_F1B_model = transfuse_weights(DA_TF_F1B_model)
  return DA_TF_F1B_model

In [ ]:
def create_first_two_blocks_architecture():
  img_input = layers.Input(shape=IMG_SHAPE) #input layer
  x = img_input
  for i, layer in enumerate(base_model.layers):
    if i >= 1 and i<= 12:
      x = layer(x)
      if i == 6:
        temp1 = x
    elif i == 13:
      shortcut1 = layer(temp1)
    elif i == 14:
      x = layer(x)
    elif i == 15:
      shortcut1 = layer(shortcut1)
    elif i == 16:
      x = layer(x)
    elif i == 17:
      x = layers.add([x, shortcut1])
    elif i >= 18 and i <= 26:
      x = layer(x)
      if i == 18:
        temp2 = x
    elif i == 27:
      x = layers.add([x, temp2])
    elif i >= 28 and i <= 36:
      x = layer(x)
      if i == 28:
        temp3 = x
    elif i == 37:
      x = layers.add([x, temp3]) 
    elif i==38:
      x = layer(x)
    elif i >= 38 and i <= 44:
      x = layer(x)
      if i == 38:
        temp4 = x
    elif i == 45:
      shortcut2 = layer(temp4)
    elif i == 46: 
      x = layer(x)
    elif i == 47:
      shortcut2 = layer(shortcut2)
    elif i == 48:
      x = layer(x)
    elif i == 49:
      x = layers.add([x, shortcut2])
    elif i >= 50 and i <= 58:
      x = layer(x)
      if i == 50:
        temp5 = x
    elif i == 59:
      x = layers.add([x, temp5])
    elif i >= 60 and i <= 68:
      x = layer(x)
      if i == 60:
        temp6 = x
    elif i == 69:
      x = layers.add([x, temp6])
    elif i >= 70 and i <= 78:
      x = layer(x)
      if i == 70:
        temp7 = x
    elif i == 79:
      x = layers.add([x, temp7])
    elif i == 80:
      x = layer(x)
  
  x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
  x = layers.Dense(1, activation='sigmoid', name='fc')(x)
  DA_TF_F2B_model = models.Model(img_input, x, name='DA_TF_F2B')

  DA_TF_F2B_model = transfuse_weights(DA_TF_F2B_model)
  return DA_TF_F2B_model

In [ ]:
#This function returns the model for the given strategy
def create_architecture(strategy):

  if strategy = "DA_TF_F1B":
    final_model = create_first_one_block_architecture()
  elif strategy = "DA_TF_F2B":
    final_model = create_first_two_blocks_architecture()
  else:
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = layers.Dense(1, activation='sigmoid', name='fc')(x)
    final_model = models.Model(inputs=base_model.input,
                              outputs=x)

  return final_model

In [ ]:
#####################################################
#You need to run phase 1 of DAPT for all the strategies
#Please use the variable "Strategy" and assign it the 
#name of the strategy for which you are running the 
#experiment.
#####################################################
Strategy = "DA_TF_F1B" #choose your strategy 
phase_1_model = create_architecture(Strategy)

In [ ]:
un_freeze_from = -2 #freeze all the layers except the last 2
for layer in phase_1_model.layers[:un_freeze_from]:
    layer.trainable = False

In [ ]:
#Verify that all the layers of the convolution base are frozen
for j, layer in enumerate(phase_1_model.layers):
  print (j, layer.name, layer.trainable)

In [ ]:
#compile the model
phase_1_model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
              loss=losses.BinaryCrossentropy(from_logits=True),
              metrics=[metrics.BinaryAccuracy()])

In [ ]:
#Set the paths where checkpoints will be saved after every epoch
#We need to set different paths for saving DA_TF_F1B and DA_TF_F2B
#As the phase 1 DAPT for all the strategies based on full ResNet50 is the same,
#all of the checkpoints of phase 1 for these strategies are saved at one path. 
if Strategy="DA_TF_F1B" or Strategy="DA_TF_F2B":
  checkpoint_path = base_path + 'DAPT/Checkpoints/' + Strategy + '/phase_1'
else: 
  checkpoint_path = base_path + 'DAPT/Checkpoints/Full Architectures/phase_1'

#Define callback to save the model after every epoch
callbacks = []
callbacks.append(ModelCheckpoint(checkpoint_path + '/checkpoint-{epoch}.h5'))

In [ ]:
#Set the total epochs, initial epoch number and batch_size for phase 1 DAPT
total_epochs = 1000
batchSize=16
initial_epoch_number = 0

In [ ]:
#Start DAPT phase 1
history = phase_1_model.fit(TrainX, TrainY,  batch_size=batchSize, 
                    epochs=total_epochs,
                    initial_epoch=initial_epoch_number,
                    verbose=2,
                    callbacks=callbacks)

# Phase 1 of DAPT (Resume Training)
Run the following cells only when resuming DAPT (phase 1) for a particular 
strategy from a particular epoch number. **DO NOT** run the following cells if you are
starting phase 1 of DAPT from epoch no. 0.

#The most recent checkpoint will be loaded and training will be resumed from where it was interrupted.

In [ ]:
#Set the paths where checkpoints will be saved after every epoch
#We need to set different paths for saving DA_TF_F1B and DA_TF_F2B
#As the phase 1 DAPT for all the strategies based on full ResNet50 is the same,
#all of the checkpoints of phase 1 for these strategies are saved at one path. 
if Strategy="DA_TF_F1B" or Strategy="DA_TF_F2B":
  checkpoint_path = base_path + 'DAPT/Checkpoints/' + Strategy + '/phase_1'
else: 
  checkpoint_path = base_path + 'DAPT/Checkpoints/Full Architectures/phase_1'

#Define callback to save the model after every epoch
callbacks = []
callbacks.append(ModelCheckpoint(checkpoint_path + '/checkpoint-{epoch}.h5'))

In [ ]:
#Set the total epochs and batch_size for phase 1 DAPT
total_epochs = 1000
batchSize=16

In [ ]:
#set the epoch number from where the training will be resumed.
initial_epoch_number = 50 

#loading the saved checkpoint from where to resume training 
phase_1_model = models.load_model(checkpoint_path + '/checkpoint-' + str(initial_epoch_number) + '.h5')

In [ ]:
#Resume DAPT phase 1 training
history = phase_1_model.fit(TrainX, TrainY,  batch_size=batchSize, 
                    epochs=total_epochs,
                    initial_epoch=initial_epoch_number,
                    verbose=2,
                    callbacks=callbacks)

#END OF PHASE 1 DAPT